In [1]:
%reset -f

In [2]:
import gc
gc.collect()

0

In [3]:
import psutil
psutil.virtual_memory()

svmem(total=64384073728, available=63054004224, percent=2.1, used=615870464, free=51560865792, active=4153495552, inactive=8052359168, buffers=219156480, cached=11988180992, shared=73728, slab=417730560)

## Import Modules

In [4]:
import pandas as pd
import numpy as np
import os
import math
# plot module
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels.api as sm
# import lightgbm as lgb
from sklearn.metrics import confusion_matrix, classification_report
%matplotlib inline
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

## Read data

In [5]:
df = pd.read_csv('all.csv') # full dataset

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Check the data size
# import sys
# sys.getsizeof(df)

## Data Cleaning

### Drop unnecessary variables 

In [7]:
df = df.drop(['DocumentNumberFormatted', 'ATTOM_ID', 'TransID', 'SitusCounty', 'PropertyAddressZIP',	'PropertyAddressZIP4', 'E_ZIP_CODE', 'ZIP', 'ZIP4', 'dr_ForeclosureRecordingDate','TaxAssessedValueImprovements','TaxAssessedValueLand','PreviousAssessedValue',
             'TaxMarketValueImprovements','TaxMarketValueLand','TaxMarketImprovementsPerc','AssessorPriorSaleAmount', 'AssessorLastSaleDate'], axis = 1)


### Drop Nan rows in "TaxAssessedImprovementsPerc"

In [8]:
df = df.dropna(subset=['TaxAssessedImprovementsPerc'])



### 0 to null

In [9]:
nan_list = ['Mortgage1Amount', 'TaxAssessedValueTotal', 'TaxMarketValueTotal', 'RoomsCount']
for col in nan_list:
  df[col] = df[col].replace(0, np.NaN)
  df[col] = df[col].replace(0.0, np.NaN)

In [10]:
e_var = ['E_DENSITY',	'E_POP10',	'E_HH10',	'E_FAMMAR18',	'E_NONFAMHH',	'E_TRAVHOME',	'E_OOCCHH',	'E_ROCCHH',	'E_MORTANY',	'E_NOMORT',	'E_MED_BLT',	'E_POV_TOTAL',	'E_MEDAGE',	'E_MEDHHSIZE',	'E_MEDRENT',	'E_MEDVALOCC',	'E_MEDHHINC','E_SQMILES']
for i in e_var:
  df.loc[df['E_SQMILES']==0, i] = np.NaN

### Checking the portion of forclosure 0 before dropna & choose the percentage for dropping

In [11]:
#checking missing%
missing = pd.DataFrame(df.isna().sum(), columns = ['count'])
missing['%'] = round(missing/len(df),2)
missing = missing[missing['count'] != 0]

In [12]:
# ogirinal forclosure portion
df['dr_Foreclosure_Ind'].value_counts()[1]/len(df)

0.06362654136906327

In [13]:
# forclosure portion after dropping null (in variables having null below 0.7; I tired various values and 0.7 is the best)
missing_drop = missing[missing['%'] < 0.07].index.tolist()
ndf = df.dropna(subset = missing_drop)
ndf['dr_Foreclosure_Ind'].value_counts()[1]/len(ndf)

0.06755177487602111

### drop na about some columns

In [14]:
# dropping null (in variables having null below 0.7)
df = df.dropna(subset = missing_drop)

In [15]:
#checking missing% again
missing = pd.DataFrame(df.isna().sum(), columns = ['count'])
missing['%'] = round(missing/len(df),2)
missing = missing[missing['count'] != 0]

In [16]:
# Final missing value list
df[missing.index].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6474086 entries, 0 to 7237382
Data columns (total 9 columns):
Mortgage1Amount            float64
StatusOwnerOccupiedFlag    float64
AreaBuilding               float64
RoomsCount                 float64
dr_DepthToWidth            float64
dr_YearBuilt               float64
dr_Ins_Date                object
dr_Ownership_Period        float64
dr_House_Age               float64
dtypes: float64(8), object(1)
memory usage: 493.9+ MB


### Creating new columns for missing values (comebine this later)

In [17]:
# get column names having nan
nan_check = df.columns[df.isna().any()].tolist()

# make new columns to check nan
mdf = pd.DataFrame()
for i in nan_check:
  v_name = i + '_Null'
  mdf[v_name] = df[i].isnull().apply(lambda x:  1 if x == True else 0)

### Convert date to unixtime

In [18]:
def time_f(x):
    try:
        return time.mktime(datetime.datetime.strptime(x, "%Y-%m-%d").timetuple())
    except:
        return np.NaN

In [19]:
import time
import datetime


df['dr_Ins_Date'] = df['dr_Ins_Date'].astype(str)
df['dr_Ins_Date'] = df['dr_Ins_Date'].apply(lambda x: time_f(x))

In [20]:
df['dr_Ins_Date'].head()

0    1.197590e+09
1    1.286323e+09
2    1.355443e+09
3    1.349136e+09
4             NaN
Name: dr_Ins_Date, dtype: float64

### One-hot-encoding

In [21]:
df = pd.get_dummies(df, columns = ['dr_Value_Change'], prefix='dr_Value_Change')

### Combine missing check

In [22]:
df = pd.concat([df, mdf], axis=1)

### Impute Missing Value

In [23]:
#checking missing% again
missing = pd.DataFrame(df.isna().sum(), columns = ['count'])
missing['%'] = round(missing/len(df),2)
missing = missing[missing['count'] != 0]

In [24]:
# median
for i in missing.index:
    df[i] = df[i].fillna(value=df[i].median())

## Standard Scaling

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
df_sc = scaler.transform(df)
df_sc = pd.DataFrame(df_sc, columns=df.columns)

In [26]:
df_sc.head()

,Mortgage1Amount,StatusOwnerOccupiedFlag,TaxYearAssessed,TaxAssessedValueTotal,TaxAssessedImprovementsPerc,TaxMarketValueTotal,AssessorLastSaleAmount,AreaBuilding,RoomsCount,E_SQMILES,...,dr_Value_Change_NC,Mortgage1Amount_Null,StatusOwnerOccupiedFlag_Null,AreaBuilding_Null,RoomsCount_Null,dr_DepthToWidth_Null,dr_YearBuilt_Null,dr_Ins_Date_Null,dr_Ownership_Period_Null,dr_House_Age_Null
0,0.000054,1.0,1.0,0.000317,0.008070,0.000317,0.000000,0.000074,0.000603,0.000373,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,0.000005,1.0,1.0,0.000247,0.008525,0.000247,0.000476,0.000094,0.000603,0.000373,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.000144,1.0,1.0,0.000288,0.009548,0.000288,0.000535,0.000076,0.000603,0.000373,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,0.000074,1.0,1.0,0.000194,0.009093,0.000194,0.000000,0.000062,0.000603,0.000373,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.000198,1.0,1.0,0.000207,0.007843,0.000207,0.000318,0.000065,0.000603,0.000373,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


## SMOTE

In [27]:
from sklearn.model_selection import train_test_split
X = df_sc.drop('dr_Foreclosure_Ind', axis=1)
y = df_sc['dr_Foreclosure_Ind']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [28]:
y_test.value_counts()

0.0    2415146
1.0     174489
Name: dr_Foreclosure_Ind, dtype: int64

In [29]:
# !pip install imblearn

In [30]:
from imblearn.over_sampling import SMOTE
smote = SMOTE( random_state=0)

X_train, y_train = smote.fit_resample(X_train, y_train)

## Fast.AI

In [31]:
# !pip install jupyter notebook jupyter_contrib_nbextensions

In [32]:
from fastai.tabular import * 

In [33]:
procs = [FillMissing, Normalize]
valid_idx = range(len(df)-30000, len(df))
dep_var = 'dr_Foreclosure_Ind'
con = df.drop('dr_Foreclosure_Ind', axis=1).columns.tolist()

In [34]:
XX = pd.DataFrame(X_train, columns=df.drop('dr_Foreclosure_Ind', axis=1).columns)
yy = pd.DataFrame(y_train, columns=['dr_Foreclosure_Ind'])
ndf = pd.concat([XX, yy], axis=1)
test_df = pd.DataFrame(X_test, columns=df.drop('dr_Foreclosure_Ind', axis=1).columns)

In [35]:
path = '/home/ec2-user/SageMaker'

In [36]:
data = TabularDataBunch.from_df(path=path, df=ndf, 
                                dep_var=dep_var, 
                                valid_idx=valid_idx, 
                                cat_names=[], 
                                cont_names=con, 
#                                 bs=2048,
                                procs=[],
                                test_df=test_df)

In [37]:
learn = tabular_learner(data, layers=[200,100], wd=0.01, y_range=[0,1])

In [38]:
# learn.lr_find()
# learn.recorder.plot()

In [39]:
learn.fit_one_cycle(7, 1.00E-03)

epoch,train_loss,valid_loss,time
0,0.143987,0.139661,12:31
1,0.135782,0.126163,12:33
2,0.137668,0.118449,12:33
3,0.134511,0.125898,12:34
4,0.127014,0.151021,12:33
5,0.119802,0.091099,12:32
6,0.121784,0.121380,12:35


In [40]:
# learn.unfreeze()

In [41]:
# learn.fit_one_cycle(7,  max_lr=1e-2)

In [42]:
test_preds, _ = learn.get_preds(ds_type=DatasetType.Test)
target_preds = to_np(test_preds)
target_preds = target_preds.reshape((target_preds.shape[0],))
target_preds = pd.Series(target_preds)

In [64]:
prediction = target_preds.apply(lambda x: 1 if x>=.5 else 0)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,prediction))

[[2034408  380738]
 [  54059  120430]]


In [65]:
xx =confusion_matrix(y_test,prediction)[0][1] + confusion_matrix(y_test,prediction)[1][0]
yy = xx + confusion_matrix(y_test,prediction)[0][0] + confusion_matrix(y_test,prediction)[1][1]
round(1- (xx / yy),4)

0.8321

In [66]:
a1 = y_test.tolist()
a2 = prediction.tolist()

In [67]:
# data = {'Name':['Tom', 'nick', 'krish', 'jack'],
#         'Age':[20, 21, 19, 18]}

tab = {'actual': a1, 'pred': a2}

In [63]:
x1 = pd.DataFrame(tab)

In [68]:
x2 = pd.DataFrame(tab)

In [69]:
x1.to_csv('93.csv')

In [70]:
x2.to_csv('83.csv')

In [45]:
print(classification_report(prediction, y_test))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90   2088467
           1       0.69      0.24      0.36    501168

    accuracy                           0.83   2589635
   macro avg       0.77      0.61      0.63   2589635
weighted avg       0.81      0.83      0.80   2589635

